In [1]:
import numpy as np
import torch
import torch.nn as nn
import random
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [2]:
# Following is the architecture of DeepONet
# Note that it is necessary to run the code in GPU for rapid computation
# This requires to install the PyTorch that matches with your own computer's GPU
# The step is simple to follow the method of PyTorch official

num_sensors = 128
num_input = 32
std_a = 0.26688421024274595
mean_a = 0.6380248110311998

# Construct a channel attention mechanism module (more sensitive to features)

class SELayer(nn.Module):
    def __init__(self, channel, reduction=6):
        super(SELayer, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.BatchNorm1d(channel // reduction, affine=True),
            nn.LeakyReLU(0, inplace=True),
            nn.Linear(channel // reduction, channel, bias=True),
            nn.BatchNorm1d(channel, affine=True),
            nn.Sigmoid())
        self.fco = nn.Linear(channel, channel, bias=True)
        self.bn1 = nn.BatchNorm1d(channel, affine=True)

    def forward(self, input):
        y1 = self.fc1(input)
        y2 = self.bn1(self.fco(torch.mul(y1, input))+input)
        return y2
    
class SELayer_w(nn.Module):
    def __init__(self, channel, reduction=6):
        super(SELayer_w, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.BatchNorm1d(channel // reduction, affine=True),
            nn.LeakyReLU(0, inplace=True),
            nn.Linear(channel // reduction, channel, bias=True),
            nn.BatchNorm1d(channel, affine=True),
            nn.Sigmoid())
        self.fco = nn.Linear(channel, channel, bias=True)

    def forward(self, input):
        y1 = self.fc1(input)
        y2 = self.fco(torch.mul(y1, input))+input
        return y2
    
# Branch networks
# Input dimension for branch net1: [batchsize*num_input, num_sensors]
# Input dimension for branch net2: [batchsize*num_input, 1]
# Output dimension: [batchsize*num_input, self-define]
# Here self-define is suggested to be taken 64

class Branch1(nn.Module):
    # branch net1
    def __init__(self, num_sensors):
        super(Branch1, self).__init__()
        self.numinput = num_input
        self.numsensors = num_sensors
        self.net = self.__NET__(128, 4)
    
    def __NET__(self, nc, nb):
        
        layers = []
        layers.append(nn.Linear(self.numsensors, nc, bias=False))
        layers.append(nn.BatchNorm1d(nc, affine=True))
        layers.append(nn.LeakyReLU(0.02, inplace=True))
        for i in range(nb):
            layers.append(SELayer(nc))
        layers.append(nn.Linear(nc, 64, bias=True))
        return nn.Sequential(*layers)
        
    def forward(self, input):
        
        output = self.net(input)
        return output
    
class Branch2(nn.Module):
    # branch net2
    def __init__(self, num_sensors):
        super(Branch2, self).__init__()
        self.numinput = num_input
        self.numsensors = num_sensors
        self.net = self.__NET__(128, 2)
    
    def __NET__(self, nc, nb):
        
        layers = []
        layers.append(nn.Linear(1, nc, bias=False))
        layers.append(nn.BatchNorm1d(nc, affine=True))
        layers.append(nn.LeakyReLU(0.02, inplace=True))
        for i in range(nb):
            layers.append(SELayer(nc))
        layers.append(nn.Linear(nc, 64, bias=True))
        return nn.Sequential(*layers)
        
    def forward(self, input):
        
        output = self.net(input)
        return output
    
class Branch(nn.Module):
    # output of brach net1 and branch net2
    def __init__(self, num_sensors):
        super(Branch, self).__init__()
        self.branch1 = Branch1(num_sensors)
        self.branch2 = Branch2(num_sensors)
        self.se = SELayer_w(64)
    
    def _initialize_weights(self):
        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                
                nn.init.normal_(m.weight.data, 0, 0.02)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
        
    def forward(self, input1, input2):
        
        output1 = self.branch1(input1)
        output2 = self.branch2(input2)
        output = torch.mul(output1, output2)+0.001
        return self.se(output)
    
# Trunk network
# Input dimension: [batchsize*num_input, 1]
# Output dimension: [batchsize*num_input, self-define]
# Here self-define is suggested to be taken 64

class Trunk(nn.Module):
    def __init__(self):
        super(Trunk, self).__init__()
        self.net = self.__NET__(128, 1)
    
    def __NET__(self, nc, nb):
        
        layers = []
        layers.append(nn.Linear(1, nc, bias=False))
        layers.append(nn.BatchNorm1d(nc, affine=True))
        layers.append(nn.LeakyReLU(0.02, inplace=True))
        for i in range(nb):
            layers.append(SELayer(nc))
        layers.append(nn.Linear(nc, 64, bias=False))
        layers.append(nn.BatchNorm1d(64, affine=True))
        layers.append(nn.LeakyReLU(0.02, inplace=True))
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                
                nn.init.normal_(m.weight.data, 0, 0.02)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
        
    def forward(self, input):
        
        output = self.net(input)
        return output

# Read the specific parameters of the networks
# Note that the file path need to be reselected in different environment (it is simple to understand)
# We suggest retrain the network for better using in the problem that you need to solve 
# The detail training code is not provided in this file

trunk = torch.load("net_data/trunk_sigmoid_whole.pkl")
branch = torch.load("net_data/branch_sigmoid_whole.pkl")

# Close BN

branch.eval();
trunk.eval();

In [4]:
# The class for calculating the Yarkovsky database in spherical coordinate frame of convex polyhedron asteroids

# The input is

# branch: the branch networks of DeepONet

# trunk: the trunk networks of DeepONet

# para: the combination of physical parameters
# the suggested order is
# [rotation speed (s^-1)
#  Bond albedo 
#  emissivity
#  incident solar radiation flux at 1 au (W/m^2)
#  Stefan–Boltzmann constant (W m^-2 K^-4)
#  surface density (kg/m^3)
#  specific heat capacity (J K^-1 kg^-1)
#  thermal conductivity (W m^-1 K^-1)
#  length unit of facet (m)
#  quality (kg)]

# shape: the file path of txt file containing points and combinations.
# the suggested order is
# [points
#  combination methods]

class Yarkovsky_database():
    def __init__(self, branch, trunk, para, shape):
        self.branch = branch
        self.trunk = trunk
        self.para = para
        self.shape = shape
        
    def __normal__(self, f, mean, std):
        # Z-score
        
        return (f-mean)/std
    
    # Following is some computions of the triangles properties
    # A, B, C are three space point coordinates of triangles

    def __normal_vector__(self, A, B, C):
        # calculate the normal vectors of facets
        
        AB = B-A
        AC = C-A
        nv = np.cross(AB, AC)
        return nv/np.sqrt(np.sum(nv**2))

    def __center_vector__(self, A, B, C):
        # calculate the barycenter of facets
        
        Cx = (A[0]+B[0]+C[0])/3
        Cy = (A[1]+B[1]+C[1])/3
        Cz = (A[2]+B[2]+C[2])/3
        return np.array([Cx, Cy, Cz])

    def __area_surface__(self, A, B, C):
        # calculate the areas of facets
        
        v1 = B-A
        v2 = C-A
        cross_product = np.cross(v1, v2)
        area = 0.5*np.linalg.norm(cross_product)
        return area
    
    # Following is the calculations for specific asteroids
    
    def __shape_data__(self):
        # parse the input shape data
        
        point_data = np.loadtxt(self.shape[0])
        combine_method = np.loadtxt(self.shape[1])
        
        surface_element = []
        for i in range(len(combine_method)):
            surface_element.append([point_data[int(combine_method[i, 0])-1, :], point_data[int(combine_method[i, 1])-1, :], 
                                    point_data[int(combine_method[i, 2])-1, :]])
        surface_element = np.array(surface_element)

        nvs = []
        ces = []
        ars = []
        for i in range(len(surface_element)):
            nv = self.__normal_vector__(surface_element[i, 0, :], surface_element[i, 1, :], surface_element[i, 2, :])
            ce = self.__center_vector__(surface_element[i, 0, :], surface_element[i, 1, :], surface_element[i, 2, :])
            ar = self.__area_surface__(surface_element[i, 0, :], surface_element[i, 1, :], surface_element[i, 2, :])
            nvs.append(nv)
            ces.append(ce)
            ars.append(ar)
        nvs = np.array(nvs)
        ces = np.array(ces)
        ars = np.array(ars).reshape(len(surface_element), 1)

        surface = [nvs, ars]
        return surface

    def __fl__(self, beta, yita, nvs):
        # calculate the radiation flux function of every facet in spin axis coordinate system
        
        # beta: the angle between the Sun and z-axis
        # yita: the angle between the Sun and x-axis
        # nvs: the normal vectors of facets
        
        N = 128
        sun = np.array([np.sin(beta)*np.cos(yita), np.sin(beta)*np.sin(yita), np.cos(beta)])
        if np.sqrt(sun[0]**2+sun[1]**2)==0:
            lo_sun = np.arccos(0)
        else:
            lo_sun = np.arccos(sun[0]/np.sqrt(sun[0]**2+sun[1]**2))
        if yita > 180*np.pi/180:
            if np.sqrt(sun[0]**2+sun[1]**2)==0:
                lo_sun = 2*np.pi-np.arccos(0)
            else:
                lo_sun = 2*np.pi-np.arccos(sun[0]/np.sqrt(sun[0]**2+sun[1]**2))
        sun_xy = np.sqrt(sun[0]**2+sun[1]**2)
        sun = np.array([sun_xy*np.cos(lo_sun-np.linspace(0, 2*np.pi, N)), \
                        sun_xy*np.sin(lo_sun-np.linspace(0, 2*np.pi, N)), \
                        sun[2]*np.ones(N)]).T
        
        function = np.dot(nvs, sun.T)
        function[function<0] = 0

        flist_ellipsoid = interp1d(np.linspace(0, 2*np.pi, N), function, kind='linear', axis=1)

        return flist_ellipsoid
        
    def yarkovsky(self, r):
        # calculate the Yarkovsky acceleration database
        # r: the range of heliocentric distance that the database needs
        # denoted as [min, max, grid number]
        
        [min_beta, max_beta, num_b] = [0, np.pi, 128]       # here you can modify the grid numbers
        [min_yita, max_yita, num_y] = [0, 2*np.pi, 128]     # such as 100, 200, but it is suggested to be taken as 128
        [min_r, max_r, num_r] = r
        [omega, A, eps, S, sig, rho, C, kapa, R, m] = self.para
        ft = np.sqrt(rho*C*kapa)
        [nvs, ars] = self.__shape_data__()
        
        flist_ellipsoid_list = []
        for beta in np.arange(min_beta, max_beta+(max_beta-min_beta)/(num_b-1), 
                              (max_beta-min_beta)/(num_b-1)):
            flist_ellipsoid = self.__fl__(beta, 0, nvs)
            flist_ellipsoid_list.append(flist_ellipsoid)
        
        p_list = []
        Te_list = []
        for r in np.arange(min_r, max_r+(max_r-min_r)/(num_r-1), (max_r-min_r)/(num_r-1)):
            Te = np.sqrt(np.sqrt(((1-A)*S/(r**2*eps*sig))))
            phi = ft*np.sqrt(omega)/(eps*sig*Te**3)
            p_list.append(phi)
            Te_list.append(Te)
        Te_a = np.array(Te_list)
        p_test = torch.tensor(np.array(p_list)).reshape(num_r, 1).float().cuda()
        y_test = torch.zeros(1).reshape(1, 1).float().cuda()

        out2 = branch.branch2(p_test)
        out3 = trunk(y_test)

        ay_list_plot = []
        zsj = 0
        for yita in np.arange(min_yita, max_yita+(max_yita-min_yita)/(num_y-1), (max_yita-min_yita)/(num_y-1)):
            t = np.mod(np.linspace(0, 2*np.pi, 128)-yita, 2*np.pi)
            ffn_t_or = np.array([flist_ellipsoid(t) for flist_ellipsoid in flist_ellipsoid_list])
            ffn_t = torch.tensor(ffn_t_or).float().cuda()
    
            f_test = ffn_t.reshape(num_b*len(nvs), 128)
            out1 = branch.branch1(f_test)
    
            ir = 0
            for r in np.arange(min_r, max_r+(max_r-min_r)/(num_r-1), (max_r-min_r)/(num_r-1)):
                out_mid = torch.mul(out1, (out2[ir].reshape(1, 64))[:, np.newaxis, :]).reshape(num_b*len(nvs), 64)+0.001
                out = branch.se(out_mid)

                Guy = torch.mean(torch.mul(out, out3), dim=1).unsqueeze(1)+0.001
                Guy_deal = Guy.detach().cpu().numpy()*std_a+mean_a
                Results = Guy_deal.reshape(num_b, len(nvs), 1)*Te_a[ir]
        
                FF = np.sum((Results)**4*nvs*ars*R**2, axis=1)*eps*sig*2/3/(3*1e+8)
                a_Yarkovsky = FF/m
        
                ay_list_plot.append(a_Yarkovsky)
        
                if zsj%50==0:
                    print("progress：", zsj/(num_y*num_r)*100, "%")
        
                zsj = zsj+1
                ir = ir+1
        ay_plot_zsj = -np.array(ay_list_plot).transpose(1, 0, 2).reshape(num_b, num_y, num_r, 3)
        
        return ay_plot_zsj

In [5]:
# The shape data of asteroids
# The input is the file path of txt file containing points and combinations

# Following is an example for Phaethon

Phaethon1 = "test_asteroid/Phaethon.txt"
Phaethon2 = "test_asteroid/Phaethon2.txt"
shape = [Phaethon1, Phaethon2]

# The physical parameters
# The input is the combination of all parameters

# Following is an example for Phaethon

omega = 2*np.pi/(3.6*3600)
A = 0.122
eps = 0.9
S = 1357
sig = 5.67*1e-8
rho = 1500
C = 600
kapa = 0.1
R = 896.1883527847208
m = 1e+15
para = [omega, A, eps, S, sig, rho, C, kapa, R, m]

In [ ]:
# Following is an example for using the class to calculate Yarkovsky acceleration database in orbital evolution

Yd = Yarkovsky_database(branch, trunk, para, shape)
r = [0.05, 2.7, 128]

# The output is Yarkovsky acceleration 
# it consists of 
# [beta, yita, r, 3], meaning ax, ay and az in spin axis coordinate system

yarkovsky_acc = Yd.yarkovsky(r)

# The interpolation function can be ragarded as the database
# Specifically, multidimensional interpolation can be used in integration
# The detail integration code is not provided in this file